<a href="https://colab.research.google.com/github/jo-cho/eitm/blob/main/text_analysis/EPIC_RESEARCH_Keyword_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Made BY JO-CHO

아래 코드를 변경없이 모두 실행해주세요.

## import

In [3]:
# Made BY JO-CHO
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from collections import Counter
import ipywidgets as widgets
from IPython.display import display
import warnings
warnings.filterwarnings('ignore')

## load file

데이터 로딩 시간이 조금 걸릴 수 있습니다.

In [4]:
file_url = "https://github.com/jo-cho/eitm/raw/main/data/epic_data_research.xlsx"
raw_df = pd.read_excel(file_url, engine='openpyxl')

## data preprocessing

In [5]:
df = raw_df[['발간처','자료명', '발간일','요약']].sort_values('발간일').reset_index(drop=True)

# 날짜 형식을 datetime 형식으로 변환하는 함수
def convert_to_datetime(date_str):
    # 'yyyy.mm.' 형식인 경우
    if len(date_str) == 8 and date_str[-1] == '.':
        date_str += '01'
    # 'yyyy.mm.dd' 형식이 아니거나, 잘못된 형식인 경우 None 반환
    elif len(date_str) != 10 or not date_str.replace('.', '').isdigit():
        return None

    # 유효하지 않은 일자 확인 및 수정
    try:
        date = pd.to_datetime(date_str, format='%Y.%m.%d', errors='raise')
        # 유효하지 않은 월이나 일을 가진 경우 None 반환
        if date.month == 0 or date.day == 0:
            return None
    except ValueError:
        # 잘못된 날짜는 'yyyy.mm.01'로 수정
        date_str = date_str[:8] + '01'
        try:
            date = pd.to_datetime(date_str, format='%Y.%m.%d')
        except ValueError:
            return None
    return date

# '발간일' 컬럼을 datetime 형식으로 변환
df['발간일'] = df['발간일'].apply(convert_to_datetime)

## PLOT

아래 코드를 실행하면, 밑에 입력 칸이 나옵니다.

포함 단어에 추이를 알고 싶은 키워드를 입력해주세요.

여러 단어일 경우에는 콤마(,)로 구분해주세요.
(예시: 저출산, 저출생)

In [6]:
# 키워드 입력 위젯
keywords_widget = widgets.Text(
    value='단어를 입력하세요',
    description="포함 단어:",
    style={'description_width': 'initial'}
)

# 날짜 범위 입력 위젯
start_date_widget = widgets.Text(
    value='2015-01-01',
    description="시작날짜(yyyy-mm-dd):",
    style={'description_width': 'initial'}
)
end_date_widget = widgets.Text(
    value='2024-10-31',
    description='끝날짜(yyyy-mm-dd):',
    style={'description_width': 'initial'}
)

# 집계 기간 선택 위젯
period_widget = widgets.Dropdown(
    options=['daily', 'monthly', 'annually'],
    value='monthly',
    description='기간:',
    style={'description_width': 'initial'}
)

# 요약/제목 선택 위젯
from_summary_widget = widgets.Dropdown(
    options=['요약','제목'],
    value='요약',
    description='텍스트 출처:',
    style={'description_width': 'initial'}
)

# 출력 영역
output = widgets.Output()

# 위젯 디스플레이
display(keywords_widget, start_date_widget, end_date_widget, period_widget, from_summary_widget)

# 함수 정의 및 실행
def contains_keyword(text, keywords):
    if isinstance(text, str):
        return int(any(keyword in text for keyword in keywords))
    return 0

def count_by_period(data, period, from_summary='요약', start_date='1996-01-01', end_date='2024-05-31'):
    data = data[(data['발간일'] >= start_date) & (data['발간일'] <= end_date)]
    if from_summary == '요약':
        keyword_col = '포함_요약'
    elif from_summary == '제목':
        keyword_col = '포함_제목'

    if period == 'daily':
        counts = data.groupby(data['발간일'].dt.date)[keyword_col].agg(['count', 'sum'])
    elif period == 'monthly':
        counts = data.groupby(data['발간일'].dt.to_period('M'))[keyword_col].agg(['count', 'sum'])
    elif period == 'annually':
        counts = data.groupby(data['발간일'].dt.to_period('Y'))[keyword_col].agg(['count', 'sum'])
    else:
        raise ValueError("Invalid period. Choose from 'daily', 'monthly', or 'annually'.")

    counts.columns = ['total_count', 'keyword_count']
    return counts['keyword_count']

def on_button_click(b):
    with output:
        output.clear_output()

        # 입력 값 가져오기
        keywords = keywords_widget.value.split(',')
        keywords = [kw.strip() for kw in keywords]
        start_date = start_date_widget.value
        end_date = end_date_widget.value
        period = period_widget.value
        from_summary = from_summary_widget.value

        # 데이터 전처리
        df['포함_요약'] = df['요약'].apply(lambda x: contains_keyword(x, keywords))
        df['포함_제목'] = df['자료명'].apply(lambda x: contains_keyword(x, keywords))

        # 결과 계산 및 시각화
        result = count_by_period(df, period=period, from_summary=from_summary, start_date=start_date, end_date=end_date)

        plt.figure(figsize=(16, 6))
        result.plot()
        plt.title(f'Research Keywords Plot ({period})')
        plt.xlabel('Date')
        plt.ylabel('Counts')
        plt.show()

        print(f"포함 단어(또는): {keywords}")

# 버튼 생성
button = widgets.Button(
    description='그리기',
    disabled=False,
    button_style='info',
    tooltip='누르면 그래프가 그려집니다.',
)

button.on_click(on_button_click)

# 버튼 디스플레이
display(button, output)

Text(value='단어를 입력하세요', description='포함 단어:', style=DescriptionStyle(description_width='initial'))

Text(value='1996-01-01', description='시작날짜(yyyy-mm-dd):', style=DescriptionStyle(description_width='initial'))

Text(value='2024-05-31', description='끝날짜(yyyy-mm-dd):', style=DescriptionStyle(description_width='initial'))

Dropdown(description='기간:', index=1, options=('daily', 'monthly', 'annually'), style=DescriptionStyle(descript…

Dropdown(description='텍스트 출처:', options=('요약', '제목'), style=DescriptionStyle(description_width='initial'), val…

Button(button_style='info', description='그리기', style=ButtonStyle(), tooltip='누르면 그래프가 그려집니다.')

Output()

주의할 점:

- 위는 단순 키워드 빈도 수이기 때문에, 정책 흐름을 확인하는데 효과적이지 않을 수 있다. 애초에 연도별로 나오는 (혹은 구축된) 총 정책 수의 추세가 다를 수 있기 때문이다.
- 따라서, 당일/당월 나온 자료 수 중 해당 키워드 포함 자료 수의 비율로 지수를 만드는 방법이 있다. 아래에 코드가 있다.

In [7]:
# 키워드 입력 위젯
keywords_widget = widgets.Text(
    value='단어를 입력하세요',
    description="포함 단어:",
    style={'description_width': 'initial'}
)

# 날짜 범위 입력 위젯
start_date_widget = widgets.Text(
    value='2015-01-01',
    description="시작날짜(yyyy-mm-dd):",
    style={'description_width': 'initial'}
)
end_date_widget = widgets.Text(
    value='2024-10-31',
    description='끝날짜(yyyy-mm-dd):',
    style={'description_width': 'initial'}
)

# 집계 기간 선택 위젯
period_widget = widgets.Dropdown(
    options=['daily', 'monthly', 'annually'],
    value='monthly',
    description='기간:',
    style={'description_width': 'initial'}
)

# 요약/제목 선택 위젯
from_summary_widget = widgets.Dropdown(
    options=['요약','제목'],
    value='요약',
    description='텍스트 출처:',
    style={'description_width': 'initial'}
)

# 출력 영역
output = widgets.Output()

# 위젯 디스플레이
display(keywords_widget, start_date_widget, end_date_widget, period_widget, from_summary_widget)

def count_by_period_ratio(data, period, from_summary='요약', start_date='1996-01-01', end_date='2024-05-31'):
    data = data[(data['발간일'] >= start_date) & (data['발간일'] <= end_date)]
    if from_summary == '요약':
        keyword_col = '포함_요약'
    elif from_summary == '제목':
        keyword_col = '포함_제목'

    if period == 'daily':
        counts = data.groupby(data['발간일'].dt.date)[keyword_col].agg(['count', 'sum'])
    elif period == 'monthly':
        counts = data.groupby(data['발간일'].dt.to_period('M'))[keyword_col].agg(['count', 'sum'])
    elif period == 'annually':
        counts = data.groupby(data['발간일'].dt.to_period('Y'))[keyword_col].agg(['count', 'sum'])
    else:
        raise ValueError("Invalid period. Choose from 'daily', 'monthly', or 'annually'.")

    counts.columns = ['total_count', 'keyword_count']
    counts['keyword_ratio'] = counts['keyword_count'] / counts['total_count']
    return counts['keyword_ratio']

def on_button_click(b):
    with output:
        output.clear_output()

        # 입력 값 가져오기
        keywords = keywords_widget.value.split(',')
        keywords = [kw.strip() for kw in keywords]
        start_date = start_date_widget.value
        end_date = end_date_widget.value
        period = period_widget.value
        from_summary = from_summary_widget.value

        # 데이터 전처리
        df['포함_요약'] = df['요약'].apply(lambda x: contains_keyword(x, keywords))
        df['포함_제목'] = df['자료명'].apply(lambda x: contains_keyword(x, keywords))

        # 결과 계산 및 시각화
        result = count_by_period_ratio(df, period=period, from_summary=from_summary, start_date=start_date, end_date=end_date)

        plt.figure(figsize=(16, 6))
        result.plot()
        plt.title(f'Research Keywords Ratio Plot ({period})')
        plt.xlabel('Date')
        plt.ylabel('Keyword Ratio')
        plt.show()

        print(f"포함 단어(또는): {keywords}")

# 버튼 생성
button = widgets.Button(
    description='그리기',
    disabled=False,
    button_style='info',
    tooltip='누르면 그래프가 그려집니다.',
)

button.on_click(on_button_click)

# 버튼 디스플레이
display(button, output)

Text(value='단어를 입력하세요', description='포함 단어:', style=DescriptionStyle(description_width='initial'))

Text(value='1996-01-01', description='시작날짜(yyyy-mm-dd):', style=DescriptionStyle(description_width='initial'))

Text(value='2024-05-31', description='끝날짜(yyyy-mm-dd):', style=DescriptionStyle(description_width='initial'))

Dropdown(description='기간:', index=1, options=('daily', 'monthly', 'annually'), style=DescriptionStyle(descript…

Dropdown(description='텍스트 출처:', options=('요약', '제목'), style=DescriptionStyle(description_width='initial'), val…

Button(button_style='info', description='그리기', style=ButtonStyle(), tooltip='누르면 그래프가 그려집니다.')

Output()